# A simple UKF + CSS simulation

In this notebook:
* A 3U CubeSat with 6 Coarse Sun Sensor (CSS) rotating in with constant angular velocity around local and global coordinate systems
* A Simulator to model the state-variables (position, Euler angles, velocity, and angular velocity) and sensor-observation (light intensity)

_Here we use Homogeneous Coordinates and Unit Quaternions_

In [1]:
%matplotlib inline

import numpy as np
from src.unscented_kalman_filter import UKF

import matplotlib.pyplot as plt
import seaborn as sns 

from ipywidgets import interact, interactive, IntSlider, FloatSlider, fixed

## Helpers

In [ ]:
class Simulator:
    def __init__(x0, w, w_std):
        '''
        Inputs:
        - x0: initial state
        - w: angular velocity in local coordinate system of the satellite
        - w_std: indicates the error in the angular velocity
        '''
        self.X = x0
        self.w = w
        self.w_std = w_std
        
    def step():
        # TODO
        pass

In [ ]:
def h(sensors, light_source):
    '''
    Inputs:
    - sensors: [[position, normal_vector]]
    - light_source: position
    '''
    def f(state):
        # TODO
        pass
    
    return f

In [ ]:
class Observation:
    def __init__(sensors, light_source, std, forward_model):
        '''
        Inputs:
        - sensors: [[position, normal_vector]]
        - light_source: position
        - std: indicates the error in the sensor observation
        - forward_model: a function that maps from state-space to observation-space
        '''
        self.h = h(sensors, light_source)
        self.std = std
        
    def measure(state):
        z = self.h(state)
        return z + np.random.randn(*z.shape) * self.std